# Collect Transactions

## Set up

In [1]:

import os
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import shutil

PATH = Path.cwd().parents[0]

from utils.data_loader import (
  get_info_by_contract_addr,
  save_bytecode_by_contract_addr,
  get_bytecode_by_contract_addr,
  save_transactions_by_contract_addr,
  get_source_code_by_contract_addr,
  save_sol_by_contract_addr
)

In [2]:
DATA_PATH = os.path.join(PATH, 'data')
LABELED_PATH = os.path.join(DATA_PATH, 'labeled')
TXN_PATH = os.path.join(LABELED_PATH, 'txn')
HEX_PATH = os.path.join(LABELED_PATH, 'hex')
SOL_PATH = os.path.join(LABELED_PATH, 'sol')
GROUND_PATH = os.path.join(DATA_PATH, 'external/crpwarner/dataset/groundtruth')
HEX_GROUND_PATH = os.path.join(GROUND_PATH, 'hex')
SOL_GROUND_PATH = os.path.join(GROUND_PATH, 'sol')
TMP_PATH = os.path.join(DATA_PATH, 'interim')
HEX_TMP_PATH = os.path.join(TMP_PATH, 'hex')
SOL_TMP_PATH = os.path.join(TMP_PATH, 'sol')


## Load Dataset

In [3]:
df = pd.read_excel(os.path.join(GROUND_PATH, 'groundTruth.xlsx')).rename(columns={
  'address': 'Address'
})
df['Address'] = df['Address'].str.lower()

In [4]:
df.to_csv(os.path.join(LABELED_PATH, 'groundtruth.csv'), index=False)

In [5]:
contract_addresses = df['Address'].unique().tolist()

In [6]:
len(contract_addresses), contract_addresses

(69,
 ['0x93023f1d3525e273f291b6f76d2f5027a39bf302',
  '0x2753dce37a7edb052a77832039bcc9aa49ad8b25',
  '0x94b7d24552933f50a5a5705c446528806dcea381',
  '0xe0b9d4146aad6936cbfcbe4dae47e34aab96b093',
  '0x10f6f2b97f3ab29583d9d38babf2994df7220c21',
  '0x11cbc781dadaad13fc3a361772c80b1c027820af',
  '0x3e597ea168a85aa2ae5e2c4333665bcd875ed10f',
  '0x292e89d5d5bdab3af2f5838c194c1983f0140b43',
  '0xef20505c8b343d12da174bf9d8495c1ce2670989',
  '0xc71d244f7ad6c869ecbf13cbd9acae31718be4f8',
  '0x5927b72440d8a8b8c6ca5a8be60e88975f9063fc',
  '0xc709878167ed069aea15fd0bd4e9758ceb4da193',
  '0x1e4402fa427a7a835fc64ea6d051404ce767a569',
  '0xcb6cd204d783dc8d66896a6def5867d332228d7b',
  '0x455dedacbe41c178953119847f2b95e2d9ad0a1d',
  '0x831467b7b6bf9c705dc87899d48b57ee55c8d5cc',
  '0x6609f543d38816116fa5b9a98c918ca947f5455d',
  '0x90f75ca026add95ae15ecbf48efc77ed272945be',
  '0x52e4339b4b9ff254738d6e971e83440f60dc029c',
  '0x51c5807dd8398aedfcc91e6483417838b41eaeb8',
  '0xa0ffc741f109159ee203424a299e6d

## Load API Key

In [7]:
for src_file in tqdm(list(Path(HEX_GROUND_PATH).glob('*.hex'))):
    address = src_file.stem.lower()
    if address in contract_addresses:
        shutil.copy(src_file, os.path.join(HEX_PATH, f'{address}.hex'))
    else:
        shutil.copy(src_file, os.path.join(HEX_TMP_PATH, f'{address}.hex'))

  0%|          | 0/70 [00:00<?, ?it/s]

100%|██████████| 70/70 [00:00<00:00, 390.01it/s]


In [8]:
for src_file in tqdm(list(Path(SOL_GROUND_PATH).glob('*.sol'))):
    address = src_file.stem.lower()
    if address in contract_addresses:
        shutil.copy(src_file, os.path.join(SOL_PATH, f'{address}.sol'))
    else:
        shutil.copy(src_file, os.path.join(SOL_TMP_PATH, f'{address}.sol'))

100%|██████████| 71/71 [00:00<00:00, 462.24it/s]


In [9]:
# 3. Collect and extract
for address in tqdm(contract_addresses):
    addr = address.lower()
    if addr not in [file.stem.lower() for file in Path(TXN_PATH).glob("*.json")]:
        info = get_info_by_contract_addr(address)
        save_transactions_by_contract_addr(TXN_PATH, address, info)
        if 'creationBytecode' in info.get('creator'):
            save_bytecode_by_contract_addr(HEX_PATH, address, info['creator']['creationBytecode'])

    if addr not in [file.stem.lower() for file in Path(HEX_PATH).glob("*.hex")]:
        bytecode = get_bytecode_by_contract_addr(addr)
        save_bytecode_by_contract_addr(HEX_PATH, address, bytecode)

    if addr not in [file.stem.lower() for file in Path(SOL_PATH).glob("*.hex")]:
        source = get_source_code_by_contract_addr(address)
        if 'SourceCode' in source:
            save_sol_by_contract_addr(SOL_PATH, address, source['SourceCode'])

  0%|          | 0/69 [00:00<?, ?it/s]

  1%|▏         | 1/69 [00:00<00:36,  1.85it/s]

Saved 0x93023f1d3525e273f291b6f76d2f5027a39bf302.sol


  3%|▎         | 2/69 [00:00<00:32,  2.03it/s]

Saved 0x2753dce37a7edb052a77832039bcc9aa49ad8b25.sol


...

Saved 0xf19308f923582a6f7c465e5ce7a9dc1bec6665b1.sol


 99%|█████████▊| 68/69 [00:37<00:00,  2.06it/s]

Saved 0x87230146e138d3f296a9a77e497a2a83012e9bc5.sol


100%|██████████| 69/69 [00:38<00:00,  1.79it/s]

Saved 0x82902c20c5826984588dcd2dfcc322e05dcc435c.sol


In [10]:
len(list(Path(TXN_PATH).glob("*.json"))), len(list(Path(HEX_PATH).glob("*.hex"))), len(list(Path(SOL_PATH).glob("*.sol")))

(69, 69, 69)